In [1]:
from tropy.learn import fit_classifier, fit_classifier_onevsall, _inrad_eigenpair, predict_onevsall
from tropy.metrics import accuracy_multiple, veronese_feature_names, print_features_per_class
from tropy.ops import veronese
from tropy.utils import simplex_lattice_points
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(precision=3, suppress=True)

In [2]:
lattice_points, features = None, None
def class_df(class_name, size = None):
  global lattice_points, features
  df_class = pd.read_csv(class_name, delimiter=';', dtype=float).loc[:, 'fixed acidity':'alcohol']
  features = df_class.columns.to_list()
  df_class = MinMaxScaler().fit_transform(df_class)
  df_train, df_test = train_test_split(df_class, test_size=0.2, random_state=43)
  Ctrain, Ctest = df_train.T, df_test.T

  d = Ctrain.shape[0]
  lattice_points = list(simplex_lattice_points(d, size))
  Ctrain, Ctest = veronese(lattice_points, Ctrain), veronese(lattice_points, Ctest)

  print(Ctrain.shape, Ctest.shape)
  return Ctrain, Ctest

classes = ["./data/winequality-red.csv", "./data/winequality-white.csv"]

In [3]:
size = 2

In [4]:
Clist_train = []
Clist_test = []
for class_name in classes:
  train, test = class_df(class_name, size)
  Clist_train.append(train)
  Clist_test.append(test)

classes = ["Red Wine", "White Wine"]

(121, 1279) (121, 320)
(121, 3918) (121, 980)


In [5]:
x, l = _inrad_eigenpair(Clist_train, N=15)
print("Apex:", x)
print("Eigval:", l)

100%|██████████| 15/15 [00:00<00:00, 41.95it/s]

Apex: [ 0.005  0.086 -0.171 -0.179 -0.169 -0.2   -0.105  0.035 -0.133  0.322
 -0.047 -0.058 -0.087 -0.141 -0.056 -0.024  0.314 -0.143  0.243 -0.062
  0.034 -0.116  0.066 -0.111  0.175 -0.017  0.285 -0.385 -0.227 -0.098
  0.069  0.056 -0.317  0.067 -0.245 -0.28  -0.257  0.039  0.023  0.147
 -0.067 -0.212  0.089 -0.119  0.116 -0.155  0.097 -0.209  0.107  0.052
 -0.215  0.056  0.08   0.322  0.157  0.005  0.086 -0.171 -0.179 -0.169
 -0.2   -0.105  0.035 -0.133  0.322 -0.047 -0.058 -0.087 -0.141 -0.056
 -0.024  0.314 -0.143  0.243 -0.062  0.034 -0.116  0.066 -0.111  0.175
 -0.017  0.285 -0.385 -0.227 -0.098  0.069  0.056 -0.317  0.067 -0.245
 -0.28  -0.257  0.039  0.023  0.147 -0.067 -0.212  0.089 -0.119  0.116
 -0.155  0.097 -0.209  0.107  0.052 -0.215  0.056  0.08   0.322  0.157
  0.322  0.541  0.208  0.109  0.027 -0.026  0.2    0.109  0.691  0.3
  0.641]
Eigval: 2.2462120495170268e-05


In [6]:
predictor, sector_indicator = fit_classifier(Clist_train, x)
indicators, apices = fit_classifier_onevsall(Clist_train)

100%|██████████| 50/50 [00:01<00:00, 41.77it/s]


In [7]:
accuracy = accuracy_multiple(predictor, Clist_test)
accuracy_one_vs_all = accuracy_multiple(predict_onevsall(indicators, apices, Clist_train), Clist_test)
print(f"Accuracy: {round(accuracy, 3)}")
print(f"Accuracy (one-vs-all): {round(accuracy_one_vs_all, 3)}")

Accuracy: 0.99
Accuracy (one-vs-all): 0.995


In [8]:
if size == None:
  print_features_per_class(classes, features, sector_indicator)
else:
  print_features_per_class(classes, veronese_feature_names(features, lattice_points), sector_indicator)

Dominant features for each class:
- Red Wine: fixed acidity + citric acid, fixed acidity + free sulfur dioxide, fixed acidity + density, volatile acidity + chlorides, volatile acidity + free sulfur dioxide, volatile acidity + density, volatile acidity + pH, citric acid + residual sugar, citric acid + free sulfur dioxide, citric acid + total sulfur dioxide, citric acid + density, citric acid + sulphates, residual sugar + free sulfur dioxide, residual sugar + total sulfur dioxide, residual sugar + pH, chlorides + density, free sulfur dioxide + density, free sulfur dioxide + pH, free sulfur dioxide + alcohol, total sulfur dioxide + density, density + pH, density + sulphates, density + alcohol, 2*volatile acidity, 2*residual sugar, 2*free sulfur dioxide, 2*total sulfur dioxide, 2*density
- White Wine: fixed acidity + volatile acidity, fixed acidity + residual sugar, fixed acidity + chlorides, fixed acidity + total sulfur dioxide, fixed acidity + pH, fixed acidity + sulphates, fixed acidity